In [76]:
campaign_id = 2
manager_ethereum_address = '0x3708045C19c4D0eaB69289F8BC10D4a1fF4EDc6a'

In [77]:
import base64
import mysql.connector as mysql
import datetime
import pandas as pd
import time
import requests

import json
from web3 import Web3 # https://web3py.readthedocs.io/en/stable/contracts.html#contract-functions

#######
## INIT
#######

# load api key
secret = {}
with open('secret.txt') as f:
    lines = f.readlines()
    for line in lines:
        secret[line.split("=")[0]] = line.split("=")[1].replace("\n","")

# init web3
infura_url = secret['INFURAURL1']
web3 = Web3(Web3.HTTPProvider(infura_url))
print(f"Connected to infura: {infura_url}")

# load abi
with open('abi.json') as f:
    abi = json.load(f)

# load bytecode
with open('bytecode.txt', 'r') as file:
    bytecode = file.read().replace('\n', '')

# load contract address
contract_address = secret['CONTRACTADDRESS']

Connected to infura: https://mainnet.infura.io/v3/1070d8486bc64afdb9c730d6f2850d6c


In [78]:
###########################
## GET USER:WALLET MAPPINGS
###########################

# convert rewards pending twitter handles to wallet ids
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT twitter_handle, ethereum_address FROM users;'
cursor.execute(query)
records = cursor.fetchall()
cursor.close()
db.close()
# convert to dataframe
users = []
for record in records:
    users.append(dict(zip(['twitter_handle', 'ethereum_address'], record)))
# convert to dataframe and lowercase handle
users = pd.DataFrame(users)
users['twitter_handle'] = users['twitter_handle'].str.lower()

# create dicts
user_wallets = dict(zip(list(users['twitter_handle']), users['ethereum_address']))
wallets_users = dict(zip(list(users['ethereum_address']), users['twitter_handle']))

# create list
user_list = list(user_wallets.keys())

In [86]:
##################################################
### get all rewards for this campaign as dataframe
##################################################
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT campaign_id, twitter_handle FROM rewards WHERE blockchain_write_time is NULL and manager_ethereum_address = %s and campaign_id = %s;'
values = (manager_ethereum_address, campaign_id)
cursor.execute(query, values)
records_rewards = cursor.fetchall()
cursor.close()
db.close()
# convert to dataframe
rewards = []
for record in records_rewards:
    rewards.append(dict(zip(['campaign_id', 'twitter_handle'], record)))
df = pd.DataFrame(rewards)
print(f"DEGBUG: LEN DF {len(df)}")

DEGBUG: LEN DF 20


In [87]:
###################################################
### GET LIST OF WALLETS TO REWARD FOR THIS CAMPAIGN
###################################################

# list of handles to reward
rewards_pending = list(df[df['campaign_id'] == campaign_id]['twitter_handle'])

# convert from twitter handles to wallets
rewards_pending = [user_wallets[r] for r in rewards_pending]

In [88]:
print(f"{len(rewards_pending)} rewards_pending")
print(rewards_pending)

20 rewards_pending
['0x4664FDA0BfE38D9C43e4997A0af5D5899527EB0b', '0x31EFE0Abf3840f78cfFf414b44A2846fe13e4688', '0x7e30C4cfFDbe6df08cD258Ee723d1fcd8D234b4f', '0x9fb470AE1278BE58fe7511F465812D44696F98f1', '0x98BCe2Cc31c3C05c6d28EEcDB1476E7A133ea563', '0xde7245952B0fc83f4d098712DEe174fA54DF0e2c', '0xB76b1011e6E64Ec129B161f188509D972C852Dcc', '0xe5D4c76Ee57f112f7e579c8D76EE7F975f5E64e4', '0x03e10c7B9F18dC8ec29B622Ede6A60211Bb801e5', '0x743DAace757269Cf3e3f57F1DF139117308849ff', '0x49d56C2020cAAe817C3b493B00E5c6dEC02f140C', '0x7B7f84ce1D67a5fA47835B36552CC6973964Ba24', '0xDdAaC482530e2d5C31C19727c6721e192d539666', '0xB2b03244224D40432f1Fb9B58468403C9027A576', '0xf3E3a1f4a01a093183F4468CC0d80cdE9B03B2CD', '0xa8672C3cef0dCe305102f20AC820c921c91fF7e2', '0x71ce21a0776DF966052cAc4FE98d18313dd3c214', '0x5bDC20C8D3b00268c1CB0D1Ae404956B6feEf19A', '0x70D9C0d1dA746c7849E4aca3a3b2cA406a478869', '0xe77f4ad2D5d75c91B94fb7829171ef44335CeC0D']


In [89]:
# # MANUAL OVERRIDE FOR DEBUGGING - COMMENT OUT BELOW FOR REAL SC WRITE
# rewards_pending = []
# rewards_pending.append('0x3c387EcA197D3E30b5b938Fac8BdA96F30c8c5BF') # alxcnwy

In [91]:
for i, reward_pending in enumerate(rewards_pending):
    print(f"Creating SC tx {i+1}/{len(rewards_pending)} to reward {reward_pending}")

    ###################################
    ### WRITE REWARDS TO SMART CONTRACT
    ###################################
    
    # get gas price from ethgasstation.info
    gasPrice = 25
    try:
        headers = {'User-Agent': 'blah',}
        response = requests.get('https://ethgasstation.info/json/ethgasAPI.json', headers=headers)
        gasPrice = int(response.json()['safeLow']/10)
        print(f"GasPrice fetched from ethgasstation.info {gasPrice}")
    except Exception as e:
        print("ERROR FETCHING GAS PRICE - using default of 25")
        print(e)

    # OVERRIDE
    gasPrice = 25

    contract = web3.eth.contract(abi=abi, bytecode=bytecode)

    tx = contract.functions.rewardAddresses(manager_ethereum_address, campaign_id, [reward_pending]).buildTransaction(
        {'gas':250000,
         'gasPrice': web3.toWei(gasPrice, 'gwei'),
         'from': secret['ETHBACKENDPUBLIC'],
         'to': contract_address,
         'nonce': web3.eth.getTransactionCount(secret['ETHBACKENDPUBLIC'])
        })

    signed_txn = web3.eth.account.signTransaction(tx, private_key=secret['ETHBACKENDPRIVATE'])
    tx_hash = web3.eth.sendRawTransaction(signed_txn.rawTransaction)
    print(f"TX hash: {tx_hash.hex()}")
    receipt = web3.eth.waitForTransactionReceipt(tx_hash, timeout=3600)
    print(receipt)

    print(f"LOGGING: {tx_hash}, rewarding: {reward_pending}, manager: {manager_ethereum_address}, ")

    if receipt['status'] == 1:
        print(f"SUCCESS writing rewards to SC for campaign {campaign_id} tx: {tx_hash.hex()}")

        # update blockchain_write_time
        db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
        cursor = db.cursor()
        query = "UPDATE rewards SET blockchain_write_time=%s, blockchain_write_tx_hash=%s, gas_used=%s, blockchain_write_tx_status=%s WHERE campaign_id=%s AND manager_ethereum_address=%s AND twitter_handle=%s;"
        values = (str(datetime.datetime.now()).split('.')[0], str(tx_hash.hex()), receipt['gasUsed'], 1, str(campaign_id), manager_ethereum_address, wallets_users[reward_pending])
        cursor.execute(query, values)
        db.commit()
        print(cursor.rowcount, "record updated")
        cursor.close()
        db.close()  
    else:
        print(f"ERROR FAIL writing rewards to SC for campaign {campaign_id} tx: {tx_hash.hex()}")

Creating SC tx 1/20 to reward 0x4664FDA0BfE38D9C43e4997A0af5D5899527EB0b
GasPrice fetched from ethgasstation.info 41
TX hash: 0xc771b5d3c3ca3fed586a006d4e201c28a68bead9d96b287bf9e4b15e76c7076c
AttributeDict({'blockHash': HexBytes('0xf128ac9e5466bb00feebb509dec2b24d1f6166f1011b1f0d72cca904f954da80'), 'blockNumber': 11408010, 'contractAddress': None, 'cumulativeGasUsed': 11902725, 'from': '0xe9540071278Dba4374A42a84B48c5e230328AeF7', 'gasUsed': 145280, 'logs': [AttributeDict({'address': '0xe7b8897D824C78e5C1076023508C43Eace3D2Afc', 'blockHash': HexBytes('0xf128ac9e5466bb00feebb509dec2b24d1f6166f1011b1f0d72cca904f954da80'), 'blockNumber': 11408010, 'data': '0x0000000000000000000000005caf454ba92e6f2c929df14667ee360ed9fd5b260000000000000000000000000000000000000000000000001bc16d674ec80000', 'logIndex': 293, 'removed': False, 'topics': [HexBytes('0xeeecc2fca78e5b3eaac3c78bc0db4c19d0b8d600a75bbb4911c4a5e458a515ac'), HexBytes('0x0000000000000000000000003708045c19c4d0eab69289f8bc10d4a1ff4edc6a')

1 record updated
Creating SC tx 5/20 to reward 0x98BCe2Cc31c3C05c6d28EEcDB1476E7A133ea563
GasPrice fetched from ethgasstation.info 40
TX hash: 0x0f78391ed9c804947ab24913059354b3418cef455803eed586db1dfaf4378c9e
AttributeDict({'blockHash': HexBytes('0xcd8a2043c82800ab45e87c49fdd9e1cd75197df173c9d2eec003d3ceb62b5f68'), 'blockNumber': 11408026, 'contractAddress': None, 'cumulativeGasUsed': 1287320, 'from': '0xe9540071278Dba4374A42a84B48c5e230328AeF7', 'gasUsed': 130280, 'logs': [AttributeDict({'address': '0xe7b8897D824C78e5C1076023508C43Eace3D2Afc', 'blockHash': HexBytes('0xcd8a2043c82800ab45e87c49fdd9e1cd75197df173c9d2eec003d3ceb62b5f68'), 'blockNumber': 11408026, 'data': '0x0000000000000000000000005caf454ba92e6f2c929df14667ee360ed9fd5b260000000000000000000000000000000000000000000000001bc16d674ec80000', 'logIndex': 26, 'removed': False, 'topics': [HexBytes('0xeeecc2fca78e5b3eaac3c78bc0db4c19d0b8d600a75bbb4911c4a5e458a515ac'), HexBytes('0x0000000000000000000000003708045c19c4d0eab69289f8bc1

1 record updated
Creating SC tx 9/20 to reward 0x03e10c7B9F18dC8ec29B622Ede6A60211Bb801e5
GasPrice fetched from ethgasstation.info 40
TX hash: 0x57c4c883009e29a43204616443b349223cc23225946bb96425dd4d8331a1a606
AttributeDict({'blockHash': HexBytes('0x33c2a1799895bda17bf805249da7818657b406a7177e13f5aeec8475288aec5f'), 'blockNumber': 11408037, 'contractAddress': None, 'cumulativeGasUsed': 5681631, 'from': '0xe9540071278Dba4374A42a84B48c5e230328AeF7', 'gasUsed': 145280, 'logs': [AttributeDict({'address': '0xe7b8897D824C78e5C1076023508C43Eace3D2Afc', 'blockHash': HexBytes('0x33c2a1799895bda17bf805249da7818657b406a7177e13f5aeec8475288aec5f'), 'blockNumber': 11408037, 'data': '0x0000000000000000000000005caf454ba92e6f2c929df14667ee360ed9fd5b260000000000000000000000000000000000000000000000001bc16d674ec80000', 'logIndex': 170, 'removed': False, 'topics': [HexBytes('0xeeecc2fca78e5b3eaac3c78bc0db4c19d0b8d600a75bbb4911c4a5e458a515ac'), HexBytes('0x0000000000000000000000003708045c19c4d0eab69289f8bc

1 record updated
Creating SC tx 13/20 to reward 0xDdAaC482530e2d5C31C19727c6721e192d539666
GasPrice fetched from ethgasstation.info 40
TX hash: 0xec221bd70a1160e8eaeb3b321b930f154e2305bd2be13a2d682e8c4cdd389e29
AttributeDict({'blockHash': HexBytes('0xbbdf900c3e043cb29f11cb46482251c8abaa98ceb826d5d19b9d2bfda67ea802'), 'blockNumber': 11408045, 'contractAddress': None, 'cumulativeGasUsed': 7433755, 'from': '0xe9540071278Dba4374A42a84B48c5e230328AeF7', 'gasUsed': 145280, 'logs': [AttributeDict({'address': '0xe7b8897D824C78e5C1076023508C43Eace3D2Afc', 'blockHash': HexBytes('0xbbdf900c3e043cb29f11cb46482251c8abaa98ceb826d5d19b9d2bfda67ea802'), 'blockNumber': 11408045, 'data': '0x0000000000000000000000005caf454ba92e6f2c929df14667ee360ed9fd5b260000000000000000000000000000000000000000000000001bc16d674ec80000', 'logIndex': 187, 'removed': False, 'topics': [HexBytes('0xeeecc2fca78e5b3eaac3c78bc0db4c19d0b8d600a75bbb4911c4a5e458a515ac'), HexBytes('0x0000000000000000000000003708045c19c4d0eab69289f8b

1 record updated
Creating SC tx 17/20 to reward 0x71ce21a0776DF966052cAc4FE98d18313dd3c214
GasPrice fetched from ethgasstation.info 34
TX hash: 0xfeddd4b21d2afb0d997aeb7c9d6e02e11fc506e39d8c072bde80bc4f8fc2a719
AttributeDict({'blockHash': HexBytes('0xe92efb39d76eff562e99c0585e2b0257a42ce649d2417efea3ed3b348adc84cb'), 'blockNumber': 11408053, 'contractAddress': None, 'cumulativeGasUsed': 3491884, 'from': '0xe9540071278Dba4374A42a84B48c5e230328AeF7', 'gasUsed': 145280, 'logs': [AttributeDict({'address': '0xe7b8897D824C78e5C1076023508C43Eace3D2Afc', 'blockHash': HexBytes('0xe92efb39d76eff562e99c0585e2b0257a42ce649d2417efea3ed3b348adc84cb'), 'blockNumber': 11408053, 'data': '0x0000000000000000000000005caf454ba92e6f2c929df14667ee360ed9fd5b260000000000000000000000000000000000000000000000001bc16d674ec80000', 'logIndex': 109, 'removed': False, 'topics': [HexBytes('0xeeecc2fca78e5b3eaac3c78bc0db4c19d0b8d600a75bbb4911c4a5e458a515ac'), HexBytes('0x0000000000000000000000003708045c19c4d0eab69289f8b

1 record updated


# Check confirmation status

In [59]:
##############################################################
### check confirmation status for all unconfirmed transactions
##############################################################

# get all unconfirmed txs
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT blockchain_write_tx_hash FROM rewards WHERE blockchain_write_tx_status = 0;'
cursor.execute(query)
records_txs = cursor.fetchall()
cursor.close()
db.close()
# convert to dataframe
txs = []
for record in records_txs:
    txs.append(record[0])
print(f"DEGBUG: UNCONFIRMED TXS {len(txs)}")

# check tx status and update db if confirmed
for tx in txs:
    receipt = web3.eth.getTransactionReceipt(tx)
    
    if receipt['status'] == 1:
        # update blockchain_write_time
        db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
        cursor = db.cursor()
        query = "UPDATE rewards SET blockchain_write_time=%s, blockchain_write_tx_status=%s, gas_used=%s WHERE campaign_id=%s AND manager_ethereum_address=%s AND twitter_handle=%s;"
        values = (str(datetime.datetime.now()).split('.')[0], 1, receipt['gasUsed'], str(campaign_id), manager_ethereum_address, wallets_users[reward_pending])
        cursor.execute(query, values)
        db.commit()
        print(cursor.rowcount, "record updated")
        cursor.close()
        db.close() 
    else:
        print(f"ERROR - {tx} receipt status != 1")
        print(receipt)

Rewarding 1/1 0xe9540071278Dba4374A42a84B48c5e230328AeF7
TX hash: 0xb1cee719715c454467106ffc2c6b73c6234b74f7b5486bf84c336fb49b70e618
AttributeDict({'blockHash': HexBytes('0xffb054245778a31764542448d8fb31a865865ad0cadf4a5465ee2f1ac315d557'), 'blockNumber': 22328506, 'contractAddress': None, 'cumulativeGasUsed': 22208, 'from': '0xe9540071278Dba4374A42a84B48c5e230328AeF7', 'gasUsed': 22208, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status': 1, 'to': '0xe7b8897D824C78e5C1076023508C43Eace

# Estimate gas

In [57]:
contract.address = contract_address

In [58]:
contract.address

'0xe7b8897D824C78e5C1076023508C43Eace3D2Afc'

In [59]:
tx = contract.functions.rewardAddresses(ids_manageraddresses[campaign_id], int(campaign_id), rewards_pending).buildTransaction(
    {'gas':3000000, 
     'gasPrice': web3.toWei('23', 'gwei'),
     'from': secret['ETHBACKENDPUBLIC'],
     'to': contract_address,
     'nonce': web3.eth.getTransactionCount(secret['ETHBACKENDPUBLIC'])
    })


In [62]:
tx.estimateGas()

AttributeError: 'dict' object has no attribute 'estimateGas'

In [61]:
contract

web3._utils.datatypes.Contract

In [ ]:
 web3.eth.estimateGas({
     "from"      : walletbase,       
     "nonce"     : value, 
     "to"        : contractAddr,     
     "data"      : data
})

In [70]:
contract.address

'0xe7b8897D824C78e5C1076023508C43Eace3D2Afc'

In [71]:
contract.functions.rewardAddresses(ids_manageraddresses[campaign_id], int(campaign_id), rewards_pending).estimateGas(    {'gas':3000000, 
     'gasPrice': web3.toWei('23', 'gwei'),
     'from': secret['ETHBACKENDPUBLIC'],
     'to': contract_address,
     'nonce': web3.eth.getTransactionCount(secret['ETHBACKENDPUBLIC'])
    })

# https://github.com/ethereum/web3.py/blob/master/web3/contract.py

ValueError: Cannot set to in estimateGas transaction